Each proposal should include the following sections:

Title


Introduction:

- Provide some relevant background information on the topic so that someone unfamiliar with it will be prepared to understand the rest of your proposal
- Clearly state the question you will try to answer with your project
- Identify and describe the dataset that will be used to answer the question


Preliminary exploratory data analysis:

- Demonstrate that the dataset can be read from the web into R 
- Clean and wrangle your data into a tidy format
- Using only training data, summarize the data in at least one table (this is exploratory data analysis). An example of a useful table could be one that reports the number of observations in each class, the means of the predictor variables you plan to use in your analysis and how many rows have missing data. 
- Using only training data, visualize the data with at least one plot relevant to the analysis you plan to do (this is exploratory data analysis). An example of a useful visualization could be one that compares the distributions of each of the predictor variables you plan to use in your analysis.


Methods:

- Explain how you will conduct either your data analysis and which variables/columns you will use. Note - you do not need to use all variables/columns that exist in the raw data set. In fact, that's often not a good idea. For each variable think: is this a useful variable for prediction?
- Describe at least one way that you will visualize the results


Expected outcomes and significance:

- What do you expect to find?
- What impact could such findings have?
- What future questions could this lead to?

In [7]:
library(tidyverse)



Warning message:
“One or more parsing issues, see `problems()` for details”
Rows: 164 Columns: 1
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): <!DOCTYPE html>

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


<!DOCTYPE html>
<chr>
"<html lang=""en"">"
<head>
<title>Diabetes Dataset | Kaggle</title>
"<meta charset=""utf-8"" />"
"<meta name=""robots"" content=""index, follow"" />"
"<meta name=""description"" content=""Diabetes Patients Data"" />"
"<meta name=""turbolinks-cache-control"" content=""no-cache"" />"
"<meta name=""keywords"" content=""tabular data,data visualization,diabetes,india,pandas"" />"
"<meta name=""viewport"" content=""width=device-width, initial-scale=1.0, maximum-scale=5.0, minimum-scale=1.0"">"
